In [1]:
import requests
import json

def fetch_ademe_dpe_metadata():
    # ID du jeu de données pour le DPE Logements Existants (depuis juillet 2021)
    dataset_id = "dpe-v2-logements-existants"
    base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
    
    print(f"--- Analyse de la base : {dataset_id} ---")

    try:
        # 1. Récupérer la structure (les colonnes)
        response = requests.get(base_url)
        response.raise_for_status()
        schema = response.json()['dataset']['schema']
        
        columns = [field['name'] for field in schema]
        print(f"Nombre de colonnes détectées : {len(columns)}\n")

        # 2. Récupérer les valeurs possibles pour chaque colonne
        # On utilise l'endpoint /observations avec l'argument 'facet'
        # Attention : On limite souvent l'affichage pour ne pas saturer la console
        results = {}
        
        for col in columns:
            print(f"Récupération des valeurs pour : {col}...", end="\r")
            facet_url = f"{base_url}/observations?columns={col}&size=0&facet={col}"
            
            res = requests.get(facet_url)
            if res.status_code == 200:
                data = res.json()
                # Extraction des valeurs uniques via les facettes
                if 'facets' in data and col in data['facets']:
                    values = [item['value'] for item in data['facets'][col]]
                    results[col] = values
                else:
                    results[col] = "Valeurs continues ou trop nombreuses"
            else:
                results[col] = "Erreur lors de la récupération"

        return results

    except Exception as e:
        return f"Erreur : {e}"

# Exécution
metadata = fetch_ademe_dpe_metadata()

# Affichage propre des 10 premières colonnes pour l'exemple
print("\n\n--- Aperçu des colonnes et valeurs possibles ---")
for col, vals in list(metadata.items()):
    print(f"\n[Colonne] : {col}")
    if isinstance(vals, list):
        print(f"  Valeurs : {', '.join(map(str, vals[:15]))} {'...' if len(vals) > 15 else ''}")
    else:
        print(f"  Note : {vals}")

--- Analyse de la base : dpe-v2-logements-existants ---


--- Aperçu des colonnes et valeurs possibles ---


AttributeError: 'str' object has no attribute 'items'

In [ ]:
import requests
import pandas as pd
import io

def fetch_dpe_data_direct():
    # On demande les 500 premières lignes en CSV pour avoir un échantillon représentatif
    url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
    params = {
        "format": "csv",
        "size": 500
    }
    
    print("--- Téléchargement d'un échantillon de données ---")
    
    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        # On charge les données dans un DataFrame Pandas
        df = pd.read_csv(io.StringIO(response.text), low_memory=False)
        
        results = {}
        print(f"Analyse de {len(df.columns)} colonnes trouvées...")

        for col in df.columns:
            # On récupère les valeurs uniques, on ignore les colonnes vides (NaN)
            unique_vals = df[col].dropna().unique().tolist()
            
            # On ne garde que les 20 premières valeurs pour la lisibilité
            # Et on vérifie si la colonne n'est pas juste un identifiant technique
            if len(unique_vals) > 50:
                results[col] = "Valeurs multiples (Numérique ou ID)"
            else:
                results[col] = unique_vals

        # Sauvegarde
        with open("colonnes_dpe_direct.json", "w", encoding="utf-8") as f:
            import json
            json.dump(results, f, indent=4, ensure_ascii=False)
            
        return len(df.columns)

    except Exception as e:
        print(f"Erreur : {e}")
        return 0

# Exécution
nb_cols = fetch_dpe_data_direct()
if nb_cols > 0:
    print(f"\nSuccès ! Fichier 'colonnes_dpe_direct.json' généré avec {nb_cols} colonnes.")

--- Analyse du jeu de données : dpe03existant ---
Extraction de 230 colonnes détectées...

--- TERMINÉ AVEC SUCCÈS ---
Fichier créé : 'colonnes_dpe_ademe_complet.json' avec 0 colonnes.

Aperçu des premières données :


In [3]:
import requests
import pandas as pd
import io

def fetch_dpe_data_direct():
    # On demande les 500 premières lignes en CSV pour avoir un échantillon représentatif
    url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
    params = {
        "format": "csv",
        "size": 500
    }
    
    print("--- Téléchargement d'un échantillon de données ---")
    
    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        # On charge les données dans un DataFrame Pandas
        df = pd.read_csv(io.StringIO(response.text), low_memory=False)
        
        results = {}
        print(f"Analyse de {len(df.columns)} colonnes trouvées...")

        for col in df.columns:
            # On récupère les valeurs uniques, on ignore les colonnes vides (NaN)
            unique_vals = df[col].dropna().unique().tolist()
            
            # On ne garde que les 20 premières valeurs pour la lisibilité
            # Et on vérifie si la colonne n'est pas juste un identifiant technique
            if len(unique_vals) > 50:
                results[col] = "Valeurs multiples (Numérique ou ID)"
            else:
                results[col] = unique_vals

        # Sauvegarde
        with open("colonnes_dpe_direct.json", "w", encoding="utf-8") as f:
            import json
            json.dump(results, f, indent=4, ensure_ascii=False)
            
        return len(df.columns)

    except Exception as e:
        print(f"Erreur : {e}")
        return 0

# Exécution
nb_cols = fetch_dpe_data_direct()
if nb_cols > 0:
    print(f"\nSuccès ! Fichier 'colonnes_dpe_direct.json' généré avec {nb_cols} colonnes.")

--- Téléchargement d'un échantillon de données ---
Analyse de 226 colonnes trouvées...

Succès ! Fichier 'colonnes_dpe_direct.json' généré avec 226 colonnes.


In [ ]:

URL_DPE = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"